In [ ]:
!pip install datasets
!pip install transformers==4.2.0

In [ ]:
from datasets import load_dataset

dataset = load_dataset("medical_questions_pairs")

Using custom data configuration default
Reusing dataset medical_questions_pairs (/root/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae)


In [ ]:
from transformers import AutoTokenizer, RobertaForSequenceClassification #BertForSequenceClassification
#tokenizer = AutoTokenizer.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
tokenizer = AutoTokenizer.from_pretrained("allenai/biomed_roberta_base")

#model = BertForSequenceClassification.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
model = RobertaForSequenceClassification.from_pretrained("allenai/biomed_roberta_base")

Some weights of the model checkpoint at allenai/biomed_roberta_base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allenai/biomed_roberta_base and are newly initialized: ['classi

In [ ]:
dataset = dataset["train"].train_test_split(test_size=0.2)

Loading cached split indices for dataset at /root/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-c1a85bf73591ffdb.arrow and /root/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-09efc2cdbdfb339f.arrow


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 2438
    })
    test: Dataset({
        features: ['dr_id', 'question_1', 'question_2', 'label'],
        num_rows: 610
    })
})

In [ ]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [ ]:
def tokenize(batch):
    return tokenizer(batch['question_1']+"[SEP]"+batch["question_2"], padding="max_length", truncation=True, max_length=512)

In [ ]:
train_dataset = train_dataset.map(tokenize, batched=False)
test_dataset = test_dataset.map(tokenize, batched=False)
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-b676d74d8cf74f5d.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/medical_questions_pairs/default/0.0.0/e42e9b3ce408e34f6180deae0406010a229bbbf355cb39e52b4a1bfc4a611aae/cache-87c387518672eca3.arrow


In [ ]:
len(train_dataset[0]["input_ids"])

512

In [ ]:
len(train_dataset[1]["input_ids"])

512

In [ ]:
train_dataset

Dataset({
    features: ['attention_mask', 'dr_id', 'input_ids', 'label', 'question_1', 'question_2'],
    num_rows: 2438
})

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import Trainer, TrainingArguments
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

In [ ]:
trainer.train()

Step,Training Loss
500,0.587500
1000,0.422200
1500,0.261700
2000,0.145500
2500,0.080200
3000,0.065600
3500,0.064400
4000,0.057300
4500,0.032800
5000,0.031900


RuntimeError: ignored

In [ ]:
trainer.evaluate()

Step,Training Loss
500,0.587500
1000,0.422200
1500,0.261700
2000,0.145500
2500,0.080200
3000,0.065600
3500,0.064400
4000,0.057300
4500,0.032800
5000,0.031900
